In [1]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv(r"C:\Users\pradp\OneDrive\Desktop\PythProject\predictive_maintenance.csv")
df.head()


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [9]:
df.shape


(10000, 10)

In [10]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      10000 non-null  int64  
 1   Product ID               10000 non-null  object 
 2   Type                     10000 non-null  object 
 3   Air temperature [K]      10000 non-null  float64
 4   Process temperature [K]  10000 non-null  float64
 5   Rotational speed [rpm]   10000 non-null  int64  
 6   Torque [Nm]              10000 non-null  float64
 7   Tool wear [min]          10000 non-null  int64  
 8   Target                   10000 non-null  int64  
 9   Failure Type             10000 non-null  object 
dtypes: float64(3), int64(4), object(3)
memory usage: 781.4+ KB


In [11]:
df.columns

Index(['UDI', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Target', 'Failure Type'],
      dtype='object')

In [12]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("[", "", regex=False)
    .str.replace("]", "", regex=False)
)

df.columns


Index(['udi', 'product_id', 'type', 'air_temperature_k',
       'process_temperature_k', 'rotational_speed_rpm', 'torque_nm',
       'tool_wear_min', 'target', 'failure_type'],
      dtype='object')

In [13]:
df.isnull().sum()


udi                      0
product_id               0
type                     0
air_temperature_k        0
process_temperature_k    0
rotational_speed_rpm     0
torque_nm                0
tool_wear_min            0
target                   0
failure_type             0
dtype: int64

In [14]:
df = df.dropna()
df.isnull().sum()


udi                      0
product_id               0
type                     0
air_temperature_k        0
process_temperature_k    0
rotational_speed_rpm     0
torque_nm                0
tool_wear_min            0
target                   0
failure_type             0
dtype: int64

In [15]:
df.duplicated().sum()


np.int64(0)

In [16]:
df = df.drop_duplicates()


In [17]:
df.dtypes


udi                        int64
product_id                object
type                      object
air_temperature_k        float64
process_temperature_k    float64
rotational_speed_rpm       int64
torque_nm                float64
tool_wear_min              int64
target                     int64
failure_type              object
dtype: object

In [18]:
total_records = df.shape[0]
unique_udi = df['udi'].nunique()

total_records, unique_udi


(10000, 10000)

In [19]:
df.describe()


,udi,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,target
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,300.004930,310.005560,1538.776100,39.986910,107.951000,0.033900
std,2886.89568,2.000259,1.483734,179.284096,9.968934,63.654147,0.180981
min,1.00000,295.300000,305.700000,1168.000000,3.800000,0.000000,0.000000
25%,2500.75000,298.300000,308.800000,1423.000000,33.200000,53.000000,0.000000
50%,5000.50000,300.100000,310.100000,1503.000000,40.100000,108.000000,0.000000
75%,7500.25000,301.500000,311.100000,1612.000000,46.800000,162.000000,0.000000
max,10000.00000,304.500000,313.800000,2886.000000,76.600000,253.000000,1.000000


In [20]:
df[['air_temperature_k','process_temperature_k',
    'rotational_speed_rpm','torque_nm','tool_wear_min']].mean()


air_temperature_k         300.00493
process_temperature_k     310.00556
rotational_speed_rpm     1538.77610
torque_nm                  39.98691
tool_wear_min             107.95100
dtype: float64

In [21]:
df['target'].value_counts()


target
0    9661
1     339
Name: count, dtype: int64

In [22]:
df['failure_type'].value_counts()


failure_type
No Failure                  9652
Heat Dissipation Failure     112
Power Failure                 95
Overstrain Failure            78
Tool Wear Failure             45
Random Failures               18
Name: count, dtype: int64

In [23]:
q1 = np.percentile(df['tool_wear_min'], 25)
q3 = np.percentile(df['tool_wear_min'], 75)
iqr = q3 - q1

lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

outliers = df[(df['tool_wear_min'] < lower) | (df['tool_wear_min'] > upper)]
outliers.head()


,udi,product_id,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,target,failure_type


In [24]:
outliers.shape[0]


0

In [25]:
df['tool_wear_norm'] = (
    df['tool_wear_min'] - df['tool_wear_min'].min()
) / (
    df['tool_wear_min'].max() - df['tool_wear_min'].min()
)

df[['tool_wear_min','tool_wear_norm']].head()


,tool_wear_min,tool_wear_norm
0,0,0.000000
1,3,0.011858
2,5,0.019763
3,7,0.027668
4,9,0.035573


In [26]:
df['wear_level'] = pd.qcut(
    df['tool_wear_min'],
    q=3,
    labels=['Low','Medium','High']
)

df[['tool_wear_min','wear_level']].head()


,tool_wear_min,wear_level
0,0,Low
1,3,Low
2,5,Low
3,7,Low
4,9,Low


In [27]:
df['temp_diff'] = df['process_temperature_k'] - df['air_temperature_k']
df[['air_temperature_k','process_temperature_k','temp_diff']].head()


,air_temperature_k,process_temperature_k,temp_diff
0,298.1,308.6,10.5
1,298.2,308.7,10.5
2,298.1,308.5,10.4
3,298.2,308.6,10.4
4,298.2,308.7,10.5


In [28]:
df['temp_diff'] = df['process_temperature_k'] - df['air_temperature_k']
df[['air_temperature_k','process_temperature_k','temp_diff']].head()


,air_temperature_k,process_temperature_k,temp_diff
0,298.1,308.6,10.5
1,298.2,308.7,10.5
2,298.1,308.5,10.4
3,298.2,308.6,10.4
4,298.2,308.7,10.5


In [29]:
type_summary = (
    df.groupby('type')[[
        'air_temperature_k',
        'process_temperature_k',
        'rotational_speed_rpm',
        'torque_nm',
        'tool_wear_min'
    ]]
    .mean()
    .reset_index()
)

type_summary


,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min
0,H,299.866999,309.925723,1538.147557,39.838285,107.419741
1,L,300.015833,310.012300,1539.469167,39.996600,108.378833
2,M,300.029263,310.018785,1537.598932,40.017251,107.272272


In [30]:
failure_rate = (
    df.groupby('type')['target']
    .mean()
    .reset_index()
    .rename(columns={'target':'failure_rate'})
)

failure_rate


,type,failure_rate
0,H,0.020937
1,L,0.039167
2,M,0.027694


In [31]:
df['avg_wear_by_type'] = df.groupby('type')['tool_wear_min'].transform('mean')
df['wear_deviation'] = df['tool_wear_min'] - df['avg_wear_by_type']

df[['tool_wear_min','avg_wear_by_type','wear_deviation']].head()


,tool_wear_min,avg_wear_by_type,wear_deviation
0,0,107.272272,-107.272272
1,3,108.378833,-105.378833
2,5,108.378833,-103.378833
3,7,108.378833,-101.378833
4,9,108.378833,-99.378833


In [32]:
correlation = df.select_dtypes(include=np.number).corr()
correlation


,udi,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,target,tool_wear_norm,temp_diff,avg_wear_by_type,wear_deviation
udi,1.000000,0.117428,0.324428,-0.006615,0.003207,-0.010702,-0.022892,-0.010702,0.246210,-4.449088e-03,-1.066564e-02
air_temperature_k,0.117428,1.000000,0.876107,0.022670,-0.013778,0.013853,0.082556,0.013853,-0.699583,4.945886e-03,1.381246e-02
process_temperature_k,0.324428,0.876107,1.000000,0.019277,-0.014061,0.013488,0.035946,0.013488,-0.268413,4.224617e-03,1.345309e-02
rotational_speed_rpm,-0.006615,0.022670,0.019277,1.000000,-0.875027,0.000223,-0.044188,0.000223,-0.016727,4.785291e-03,1.835806e-04
torque_nm,0.003207,-0.013778,-0.014061,-0.875027,1.000000,-0.003093,0.191321,-0.003093,0.006690,8.084260e-04,-3.099562e-03
tool_wear_min,-0.010702,0.013853,0.013488,0.000223,-0.003093,1.000000,0.105448,1.000000,-0.007689,8.256652e-03,9.999659e-01
target,-0.022892,0.082556,0.035946,-0.044188,0.191321,0.105448,1.000000,0.105448,-0.111676,3.474973e-02,1.051649e-01
tool_wear_norm,-0.010702,0.013853,0.013488,0.000223,-0.003093,1.000000,0.105448,1.000000,-0.007689,8.256652e-03,9.999659e-01
temp_diff,0.246210,-0.699583,-0.268413,-0.016727,0.006690,-0.007689,-0.111676,-0.007689,1.000000,-3.620882e-03,-7.659303e-03
avg_wear_by_type,-0.004449,0.004946,0.004225,0.004785,0.000808,0.008257,0.034750,0.008257,-0.003621,1.000000e+00,-8.971469e-15


In [33]:
correlation.style.background_gradient(cmap='coolwarm')


Matplotlib is building the font cache; this may take a moment.


,udi,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,target,tool_wear_norm,temp_diff,avg_wear_by_type,wear_deviation
udi,1.000000,0.117428,0.324428,-0.006615,0.003207,-0.010702,-0.022892,-0.010702,0.246210,-0.004449,-0.010666
air_temperature_k,0.117428,1.000000,0.876107,0.022670,-0.013778,0.013853,0.082556,0.013853,-0.699583,0.004946,0.013812
process_temperature_k,0.324428,0.876107,1.000000,0.019277,-0.014061,0.013488,0.035946,0.013488,-0.268413,0.004225,0.013453
rotational_speed_rpm,-0.006615,0.022670,0.019277,1.000000,-0.875027,0.000223,-0.044188,0.000223,-0.016727,0.004785,0.000184
torque_nm,0.003207,-0.013778,-0.014061,-0.875027,1.000000,-0.003093,0.191321,-0.003093,0.006690,0.000808,-0.003100
tool_wear_min,-0.010702,0.013853,0.013488,0.000223,-0.003093,1.000000,0.105448,1.000000,-0.007689,0.008257,0.999966
target,-0.022892,0.082556,0.035946,-0.044188,0.191321,0.105448,1.000000,0.105448,-0.111676,0.034750,0.105165
tool_wear_norm,-0.010702,0.013853,0.013488,0.000223,-0.003093,1.000000,0.105448,1.000000,-0.007689,0.008257,0.999966
temp_diff,0.246210,-0.699583,-0.268413,-0.016727,0.006690,-0.007689,-0.111676,-0.007689,1.000000,-0.003621,-0.007659
avg_wear_by_type,-0.004449,0.004946,0.004225,0.004785,0.000808,0.008257,0.034750,0.008257,-0.003621,1.000000,-0.000000


In [34]:
df.to_csv("output_predictive_data.csv", index=False)
type_summary.to_csv("type_summary.csv", index=False)
failure_rate.to_csv("failure_rate.csv", index=False)


INSIGHTS:
- Higher tool wear and temperature differences are associated with increased failure rates.
- Machine Type shows variation in average operating conditions and failure probability.
- Outlier detection and normalization help standardize sensor readings for analysis.
- Aggregated datasets were prepared for dashboarding in Power BI and Tableau.


In [36]:
import os
os.getcwd()


'C:\\Users\\pradp\\OneDrive\\Desktop\\Projects'

In [37]:
df.to_csv("./output_predictive_data.csv", index=False)
type_summary.to_csv("./type_summary.csv", index=False)
failure_rate.to_csv("./failure_rate.csv", index=False)


In [38]:
import os
os.listdir()


['.ipynb_checkpoints',
 'failure_rate.csv',
 'output_predictive_data.csv',
 'predictive_analytics.ipynb',
 'predictive_maintenance.csv',
 'predictive_maintenance.xlsx',
 'type_summary.csv']